# **Lab-11-6 PackedSequence**

이번 강의에서는 대표적인 sequential data와 길이가 각각 다른 Sequence data를 하나의 batch로 묶는 두가지 방법, padding과 packing에 대해 알아본다.

## **Sequential data**

*   **Text**
  *   "The quick brown fox jumps over the lazy dog"

    문장에서 몇 개의 단어가 있는지, 몇 개의 문자열들이 있는지는 정해져있지 않다. 이 문장에서는 9개의 단어가 있지만 다른 문장들은 더 적거나 많이 단어를 쓸 수 있다.
*   **Audio**

    Audio데이터의 경우 시간과 sampling data에 따라 audio data의 길이가 달라진다.

이와 같이 sequence data들은 길이가 미정인 data들이 많다. (반대로, 이미지의 경우 28 * 28이라던지, 가로 256, 세로 256 RGB 채널 3개와 같은 fixed size인 Tensor를 갖게 된다.)

## **Padding Method**

size가 다른 sequence data를 하나의 batch로 만들기 위해서 가장 긴 시퀀스의 길이에 맞춰 나머지 data의 뒷부분을 pad라는 token을 써서 채워 넣는다.

하나의 tensor로 표현되기 때문에 컴퓨터가 처리하기에 간편하지만 계산비용이 증가하는 단점도 있다.

## **Packing Method**

pad를 사용하는 것이 아니라 sequence의 길이를 저장하는 방식이다. 하지만 이 방법은 batch data가 길이 내림 차순으로 정렬되어있어야 작동한다.

계산비용은 적지만 구현이 padding에 비해 더 복잡하다.

![](https://github.com/dbj2000/PyTorch/raw/0e11245870a7d7cffba10726ea2a6dbf456e3e96/figures/sequence.png)

이처럼 pytorch에서 데이터를 packing, padding시킬 수 있고, 서로 변환하는 작업도 가능하다.

In [4]:
import torch
import numpy as np
from torch.nn.utils.rnn import pad_sequence, pack_sequence, pack_padded_sequence, pad_packed_sequence

## **예제 데이터**

batch size가 5이고, sequence 중 가장 긴 길이는 13인 예제 데이터이다.

In [5]:
# Random word from random word generator
data = ['hello world',
        'midnight',
        'calculation',
        'path',
        'short circuit']

# Make dictionary
char_set = ['<pad>'] + list(set(char for seq in data for char in seq)) # Get all characters and include pad token
char2idx = {char: idx for idx, char in enumerate(char_set)} # Constuct character to index dictionary
print('char_set:', char_set)
print('char_set length:', len(char_set))

char_set: ['<pad>', ' ', 'l', 'o', 'd', 'm', 't', 'u', 'g', 'i', 'c', 'e', 'h', 'n', 'a', 's', 'p', 'r', 'w']
char_set length: 19


In [6]:
# Convert character to index and make list of tensors
X = [torch.LongTensor([char2idx[char] for char in seq]) for seq in data]

# Check converted result
for sequence in X:
    print(sequence)

tensor([12, 11,  2,  2,  3,  1, 18,  3, 17,  2,  4])
tensor([ 5,  9,  4, 13,  9,  8, 12,  6])
tensor([10, 14,  2, 10,  7,  2, 14,  6,  9,  3, 13])
tensor([16, 14,  6, 12])
tensor([15, 12,  3, 17,  6,  1, 10,  9, 17, 10,  7,  9,  6])


다음과 같이 sequence의 길이가 제각각인 것을 확인할 수 있다.

In [7]:
# Make length tensor (will be used later in 'pack_padded_sequence' function)
lengths = [len(seq) for seq in X]
print('lengths:', lengths)

lengths: [11, 8, 11, 4, 13]


## **PaddedSequence (그냥 Tensor) 만들기**

주의할 점은 input이 Tensor들의 list 로 주어져야한다는 것이다. (그냥 Tensor 가 아닌 Tensor들의 list 이다.)

list 안에 있는 각각의 Tensor들의 shape가 (?, a, b, ...) 라고 할때, (여기서 ?는 각각 다른 sequence length 이다.)

pad_sequence 함수를 쓰면 (T, batch_size, a, b, ...) shape를 가지는 Tensor가 리턴된다. (여기서 T는 batch안에서 가장 큰 sequence length 이다.)

만약, pad_sequence에 명시적으로 batch_first=True라는 파라미터를 지정해주면,

(batch_size, T, a, b, ...) shape를 가지는 Tensor가 리턴된다.

기본적으로 padding 값은 0으로 되어있지만, padding_value=42와 같이 파라미터를 지정해주면, padding하는 값도 정할 수 있다.

In [8]:
# Make a Tensor of shape (Batch x Maximum_Sequence_Length)
padded_sequence = pad_sequence(X, batch_first=True) # X is now padded sequence
print(padded_sequence)
print(padded_sequence.shape)

tensor([[12, 11,  2,  2,  3,  1, 18,  3, 17,  2,  4,  0,  0],
        [ 5,  9,  4, 13,  9,  8, 12,  6,  0,  0,  0,  0,  0],
        [10, 14,  2, 10,  7,  2, 14,  6,  9,  3, 13,  0,  0],
        [16, 14,  6, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [15, 12,  3, 17,  6,  1, 10,  9, 17, 10,  7,  9,  6]])
torch.Size([5, 13])


## **PackedSequence 만들기**

먼저 input을 길이에 따른 내림차순으로 정렬한다.

In [9]:
# Sort by descending lengths
sorted_idx = sorted(range(len(lengths)), key=lengths.__getitem__, reverse=True)
sorted_X = [X[idx] for idx in sorted_idx]

# Check converted result
for sequence in sorted_X:
    print(sequence)

tensor([15, 12,  3, 17,  6,  1, 10,  9, 17, 10,  7,  9,  6])
tensor([12, 11,  2,  2,  3,  1, 18,  3, 17,  2,  4])
tensor([10, 14,  2, 10,  7,  2, 14,  6,  9,  3, 13])
tensor([ 5,  9,  4, 13,  9,  8, 12,  6])
tensor([16, 14,  6, 12])


이제 input Tensor가 정렬되었으니 pack_sequence를 이용하여 PackedSequence를 만든다.

In [10]:
packed_sequence = pack_sequence(sorted_X)
print(packed_sequence)

PackedSequence(data=tensor([15, 12, 10,  5, 16, 12, 11, 14,  9, 14,  3,  2,  2,  4,  6, 17,  2, 10,
        13, 12,  6,  3,  7,  9,  1,  1,  2,  8, 10, 18, 14, 12,  9,  3,  6,  6,
        17, 17,  9, 10,  2,  3,  7,  4, 13,  9,  6]), batch_sizes=tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1]), sorted_indices=None, unsorted_indices=None)


## **Embedding 적용해보기**

이제 RNN에 input으로 넣어서 테스트해본다.

그 전에, 위에 예제들에서는 input이 character의 index들을 가지고 있는 데이터였지만, 보통은 주로 이를 embedding한 값을 RNN의 input으로 넣어준다.

이 튜토리얼에서는 one-hot character embedding을 진행한다.

In [11]:
# one-hot embedding using PaddedSequence
eye = torch.eye(len(char_set)) # Identity matrix of shape (len(char_set), len(char_set))
embedded_tensor = eye[padded_sequence]
print(embedded_tensor.shape) # shape: (Batch_size, max_sequence_length, number_of_input_tokens)

torch.Size([5, 13, 19])


In [12]:
# one-hot embedding using PackedSequence
embedded_packed_seq = pack_sequence([eye[X[idx]] for idx in sorted_idx])
print(embedded_packed_seq.data.shape)

torch.Size([47, 19])


## **RNN 모델 만들기**

In [13]:
# declare RNN
rnn = torch.nn.RNN(input_size=len(char_set), hidden_size=30, batch_first=True)

PaddedSequence를 이용하여 RNN에 넣어본다.

In [14]:
rnn_output, hidden = rnn(embedded_tensor)
print(rnn_output.shape) # shape: (batch_size, max_seq_length, hidden_size)
print(hidden.shape)     # shape: (num_layers * num_directions, batch_size, hidden_size)

torch.Size([5, 13, 30])
torch.Size([1, 5, 30])


PackedSequence를 이용하여 RNN에 넣어본다.

In [15]:
rnn_output, hidden = rnn(embedded_packed_seq)
print(rnn_output.data.shape)
print(hidden.data.shape)

torch.Size([47, 30])
torch.Size([1, 5, 30])


### **pad_packed_sequence**
위 함수는 PackedSequence를 PaddedSequence(Tensor)로 바꾸어주는 함수이다.

PackedSequence는 각 sequence에 대한 길이 정보도 가지고있기 때문에, 이 함수는 Tensor와 함께 길이에 대한 리스트를 튜플로 리턴해준다.

리턴값: (Tensor, list_of_lengths)

In [17]:
unpacked_sequence, seq_lengths = pad_packed_sequence(embedded_packed_seq, batch_first=True)
print(unpacked_sequence.shape)
print(seq_lengths)

torch.Size([5, 13, 19])
tensor([13, 11, 11,  8,  4])


### **pack_padded_sequence**
반대로, Padding이 된 Tensor인 PaddedSequence를 PackedSequence로 바꾸어주는 함수이다.

pack_padded_sequence 함수는 실제 sequence길이에 대한 정보를 모르기때문에, 파라미터로 꼭 제공해주어야한다.

여기서 주의하여야 할 점은, input인 PaddedSequence가 길이에 따른 내림차순으로 정렬되어야 한다는 조건이 성립되어야 PackedSequence로 올바르게 변환될 수 있다.

아까 만든 padded_sequence 변수는 이 조건을 만족하지 않기 때문에 다시 새로 만든다.

In [18]:
embedded_padded_sequence = eye[pad_sequence(sorted_X, batch_first=True)]
print(embedded_padded_sequence.shape)

torch.Size([5, 13, 19])


이제 이 padding이 된 Tensor를 PackedSequence로 변환한다.

In [19]:
sorted_lengths = sorted(lengths, reverse=True)
new_packed_sequence = pack_padded_sequence(embedded_padded_sequence, sorted_lengths, batch_first=True)
print(new_packed_sequence.data.shape)
print(new_packed_sequence.batch_sizes)

torch.Size([47, 19])
tensor([5, 5, 5, 5, 4, 4, 4, 4, 3, 3, 3, 1, 1])
